## Imports

In [ ]:
import torch
import os
import json
import matplotlib.pyplot as plt

In [ ]:
import matplotlib
matplotlib.rcParams['figure.facecolor'] = 'white'

In [ ]:
%run ../iu_xray.py

In [ ]:
REPORTS_DIR = os.path.join(DATASET_DIR, 'reports')

## Preprocess reports

Clean and tokenize

### Debug tokenize functions

In [ ]:
%run ../preprocess/tokenize.py

In [ ]:
text = """findings/pneumothorax """
text_to_tokens(text)

#### Check already clean reports

Look for errors

In [ ]:
with open(os.path.join(REPORTS_DIR, 'reports.clean.v3.json'), 'r') as f:
    reports_dict = json.load(f)

In [ ]:
def search_in_reports(target):
    found = []
    for r in reports_dict.values():
        clean_text = r['clean_text']
        if re.search(target, clean_text):
            found.append({
                k: r[k]
                for k in ('filename', 'clean_text', 'findings', 'impression')
            })
    print('Found: ', len(found))
    return found

In [ ]:
search_in_reports(r'\bexample\b')

### Run preprocess

- Tokenize reports, create json with clean reports and vocabularies
- Create sentences_with_chexpert_labels.csv (takes about 12min)
- Create sentences_with_organs.csv

In [ ]:
%run -n ../preprocess/iu_xray.py

In [ ]:
reports, tokens_appearances, errors = preprocess_iu_x_ray('v4-1', [0],
                                                          override=False,
                                                          impression_fallback=False,
                                                         )
len(reports), len(tokens_appearances)

In [ ]:
vocab41 = load_vocab(REPORTS_DIR, 'v4-1')
vocab4 = load_vocab(REPORTS_DIR, 'v4')
len(vocab4), len(vocab41)

In [ ]:
df_sentences_chexpert, errors = create_sentences_with_organs()

In [ ]:
%%time

df_sentences_chexpert = create_sentences_with_chexpert_labels()
len(df_sentences_chexpert)

### Check errors

#### Check in tokens

In [ ]:
for token, n_appears in tokens.items():
    if 'NUMBER' in token:
        print(token, n_appears)

#### Check in text

In [ ]:
import re

In [ ]:
TARGET_TOKENS = ['NUMBER[^\s]']

In [ ]:
found = []
for report in reports.values():
    for token in TARGET_TOKENS:
        if re.search(token, report['clean_text']):
            found.append(report)
            
len(found)

## Rotate images

NOTE: are already rotated!!
(Run this only once)

In [ ]:
info_fname = os.path.join(DATASET_DIR, 'info.json')
with open(info_fname, 'r') as f:
    info = json.load(f)
len(info)

In [ ]:
info['marks']['rotated_left']

In [ ]:
rotations = [
    ('left', -90),
    ('right', 90),
    ('bottom', 180),
]

In [ ]:
for key, degrees in rotations:
    images_key = f'rotated_{key}'
    for image_name in info['marks'][images_key]:
        filepath = os.path.join(DATASET_DIR, 'images', image_name)
        img = Image.open(filepath).rotate(degrees)
        # img.save(filepath)

## Calculate image normalization

In [ ]:
%run ../../utils/images.py

In [ ]:
image_folder = os.path.join(DATASET_DIR, 'images')

In [ ]:
dataset = IUXRayDataset('train')
len(dataset)

In [ ]:
train_images = [
    i if i.endswith('.png') else f'{i}.png'
    for i in [r['image_name'] for r in dataset.reports]
]
len(train_images)

In [ ]:
mean, std = compute_mean_std(ImageFolderIterator(image_folder, train_images), show=True)
mean, std

### Plot average image

In [ ]:
from torchvision import transforms

In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

In [ ]:
summed = torch.zeros(3, 256, 256)

for image_name in tqdm(image_names):
    fpath = os.path.join(image_folder, image_name)
    image = transform(Image.open(fpath).convert('RGB'))
    summed += image
    
summed /= len(image_names)

In [ ]:
average_image = summed.mean(dim=0)
average_image.size()

In [ ]:
plt.imshow(average_image, cmap='gray')

## Test `IUXrayDataset` class

In [ ]:
%run ../../utils/common.py

In [ ]:
%run ../iu_xray.py

In [ ]:
dataset = IUXRayDataset(
    dataset_type='test',
    masks=True,
    masks_version='v2',
    frontal_only=True,
    image_size=(512, 512),
    seg_multilabel=False,
)
len(dataset), len(dataset.word_to_idx)

In [ ]:
item = dataset[100]
image = item.image
labels = item.labels
report = item.report
image.size(), labels.size(), len(report), \
    (isinstance(item.masks, torch.Tensor) and item.masks.size())

In [ ]:
item.masks.min(), item.masks.max()

In [ ]:
n_rows = 2
n_cols = 3

plt.figure(figsize=(n_cols*5, n_rows*5))

plt.subplot(n_rows, n_cols, 1)
plt.title(item.image_fname)
plt.imshow(tensor_to_range01(image).permute(1, 2, 0))
plt.axis('off')

if item.masks.ndim == 3:
    for index, organ in enumerate(JSRT_ORGANS):
        mask = item.masks[index]

        plt.subplot(n_rows, n_cols, index + 2)
        plt.imshow(mask)
        plt.title(organ)
        plt.axis('off')

        min_value = mask.min().item()
        max_value = mask.max().item()
        print(organ, min_value, max_value)
elif item.masks.ndim == 2:
    plt.subplot(n_rows, n_cols, 2)
    plt.imshow(item.masks)

## Plot distributions

In [ ]:
from collections import Counter

In [ ]:
%run ../iu_xray.py
%run ../common/constants.py

In [ ]:
train_dataset = IUXRayDataset('train')
val_dataset = IUXRayDataset('val')
len(train_dataset), len(val_dataset)

### Labels distribution

In [ ]:
def plot_labels_distribution(dataset):
    amounts_by_disease = sum(
        (dataset.labels_by_report[r['filename']] for r in dataset.reports),
        torch.zeros(len(CHEXPERT_DISEASES)),
    ).tolist()
    max_amount = max(amounts_by_disease)
    amounts_by_disease = list(zip(CHEXPERT_DISEASES, amounts_by_disease))
    amounts_by_disease = sorted(amounts_by_disease, key=lambda x: x[1], reverse=True)
    
    n_samples = len(dataset)
    plt.title(f'{dataset.dataset_type} (n={n_samples:,})', fontsize=20)
    plt.bar(*zip(*amounts_by_disease))
    plt.xticks(rotation=60, fontsize=15, ha='right')
    plt.ylabel('Amount of images', fontsize=18)
    plt.ylim(0, max_amount * 1.15)
    y_padding = int(max_amount * 0.03)
    
    for index, (disease, amount) in enumerate(amounts_by_disease):
        amount = int(amount)
        perc = amount / n_samples * 100
        plt.text(index, amount + y_padding, f'{amount:,}\n{perc:.0f}%', ha='center')

In [ ]:
n_rows = 1
n_cols = 2
plt.figure(figsize=(15, 5))

plt.subplot(n_rows, n_cols, 1)
plot_labels_distribution(train_dataset)

plt.subplot(n_rows, n_cols, 2)
plot_labels_distribution(val_dataset)

### Report lengths distribution

In [ ]:
def plot_n_words_distribution(dataset):
    lengths = [len(r['tokens_idxs']) for r in dataset.reports]
    plt.title(f'Report-lengths ({dataset.dataset_type}, total={len(dataset):,})')
    plt.ylabel('Amount of images')
    plt.xlabel('Number of words')
    _ = plt.hist(lengths, bins=25, range=(0, 150))

In [ ]:
n_rows = 1
n_cols = 2
plt.figure(figsize=(15, 5))

plt.subplot(n_rows, n_cols, 1)
plot_n_words_distribution(train_dataset)

plt.subplot(n_rows, n_cols, 2)
plot_n_words_distribution(val_dataset)

### Frontal vs lateral distribution

In [ ]:
def plot_view_position_distribution(dataset):
    def _reduce_pos(position):
        return position.replace('-left', '').replace('-right', '')
    positions = Counter([_reduce_pos(r['position']) for r in dataset.reports])
    
    plt.title(f'Frontal vs lateral ({dataset.dataset_type})', fontsize=20)
    plt.ylabel('Amount of images', fontsize=15)
    plt.xticks(fontsize=15)
    
    positions = sorted(positions.items(), key=lambda x: x[1], reverse=True)
    keys, values = zip(*positions)
    plt.bar(keys, values, width=0.2)
    
    plt.ylim(0, max(values) * 1.2)
    y_padding = max(values) * 0.03
    n_samples = len(dataset)
    for index, value in enumerate(values):
        perc = value / n_samples * 100
        text = f'{value:,}\n{perc:.2f}%'
        plt.text(index, value + y_padding, text, ha='center')

In [ ]:
n_rows = 1
n_cols = 2
plt.figure(figsize=(15, 5))

plt.subplot(n_rows, n_cols, 1)
plot_view_position_distribution(train_dataset)

plt.subplot(n_rows, n_cols, 2)
plot_view_position_distribution(val_dataset)

## Inspect different vocabs

In [ ]:
%run ../vocab/__init__.py

In [ ]:
vocab_full = load_vocab('iu_xray')
vocab_1 = load_vocab('iu_xray', 1)
vocab_10 = load_vocab('iu_xray', 10)
len(vocab_full), len(vocab_1), len(vocab_10)

In [ ]:
assert set(vocab_10).issubset(vocab_1)
assert set(vocab_1).issubset(vocab_full)

In [ ]:
out_of_vocab_1 = set(vocab_full) - set(vocab_1)
out_of_vocab_10 = set(vocab_1) - set(vocab_10)
len(out_of_vocab_1), len(out_of_vocab_10)

In [ ]:
out_of_vocab_10

## Get sample reports

For LATINX in AI workshop

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
from pycocoevalcap.bleu import bleu_scorer
from pycocoevalcap.rouge import rouge

In [ ]:
%run ../common.py
%run ../iu_xray.py
%run ../../utils/nlp.py
%run ../../utils/__init__.py

In [ ]:
CONSTANT_REPORT = """the heart is normal in size . the mediastinum is unremarkable . 
the lungs are clear .
there is no pneumothorax or pleural effusion . no focal airspace disease .
no pleural effusion or pneumothorax ."""

In [ ]:
dataset = IUXRayDataset(dataset_type='all')
report_reader = ReportReader(dataset.get_vocab())
len(dataset)

In [ ]:
idx = GT_IDX
item = dataset[idx]
image = arr_to_range(item.image.permute(1, 2, 0))
report_base = report_reader.idx_to_text(item.report)
plt.imshow(image)
plt.axis('off')
print(report_base)

In [ ]:
GT_IDX = 7289

In [ ]:
target = [
    'the cardiac silhouette is enlarged',
    # 'the lungs are hyper',
    # 'the heart is',
]
not_target = [
    # 'the lungs are clear',
#     'the mediastinum is unremarkable',
#     'the mediastinum is stable',
#     'the mediastinum is normal',
#     'the mediastinum is within normal limits',
]
found = []
found_names = set()
for idx, report in enumerate(dataset.reports):
    filename = report['filename']
    report = report_reader.idx_to_text(report['tokens_idxs'])
    if all(t in report for t in target) and all(t not in report for t in not_target):
        if filename not in found_names:
            found.append((idx, report))
        found_names.add(filename)
len(found)

In [ ]:
found[5]

In [ ]:
gen = 'the heart is enlarged. the mediastinum is unremarkable . the lungs are hyperinflated with mildly coarsened interstitial markings . '
# the lungs are hyperexpanded
# the lungs are hyperinflated with mildly coarsened interstitial markings
# the lungs are hyperinflated with biapical pleural-parenchymal scarring and upward retraction of the xxxx

In [ ]:
def measure_bleu_rouge(gen, gt):
    scorer = bleu_scorer.BleuScorer(n=4)
    scorer += (gen, [gt])
    bleu_1_4, _ = scorer.compute_score()
    
    scorer = rouge.Rouge()
    rouge_score = scorer.calc_score([gen], [gt])
    
    print('BLEU 1-4: ', bleu_1_4)
    print('BLEU: ', np.mean(bleu_1_4))
    print('ROUGE-L: ', rouge_score)

In [ ]:
report_1 = """the heart is normal in size . the mediastinum is unremarkable . 
the lungs are clear ."""
report_2 = """the heart is normal . the mediastinum is otherwise unremarkable . 
lungs are both clear ."""
measure_bleu_rouge(report_1, report_2)

In [ ]:
report = report_reader.idx_to_text(dataset[GT_IDX].report)
report

In [ ]:
gt = """the cardiac silhouette is enlarged .
the lungs are hyperexpanded with flattening of the bilateral hemidiaphragms .
no pneumothorax or pleural effusion ."""
# the lungs are hyperinflated with mildly coarsened interstitial markings .
# with flattening of the bilateral hemidiaphragms 

In [ ]:
gen = """the cardiac silhouette is normal in size .
the lungs are clear .
no pneumothorax or pleural effusion ."""

In [ ]:
measure_bleu_rouge(gen, gt)

In [ ]:
gt = "the cardiac silhouette is enlarged . the lungs are hyperexpanded with flattening of the bilateral hemidiaphragms . no pneumothorax or pleural effusion ."
gen = "the cardiac silhouette is normal in size and configuration . the lungs are clear . no pneumothorax or pleural effusion ."
measure_bleu_rouge(gen, gt)

In [ ]:
measure_bleu_rouge(gen, gt)

## Check no-findings vs labels==0

In [ ]:
from collections import defaultdict

In [ ]:
chexpert_path = os.path.join(REPORTS_DIR, 'reports_with_chexpert_labels.csv')
mirqi_path = os.path.join(REPORTS_DIR, 'reports_with_mirqi_labels.csv')

In [ ]:
chexpert_df = pd.read_csv(chexpert_path, index_col=0)
chexpert_df.replace(-1, 1, inplace=True)
chexpert_df.replace(-2, 0, inplace=True)
chexpert_df.head()

In [ ]:
mirqi_df = pd.read_csv(mirqi_path, index_col=0)
mirqi_df.drop(columns=['attributes-gen', 'MIRQI-r', 'MIRQI-p', 'MIRQI-f'], inplace=True)
mirqi_df.rename(columns={'attributes-gt': 'attributes'}, inplace=True)
mirqi_df.replace(-1, 1, inplace=True)
mirqi_df.replace(-2, 0, inplace=True)
mirqi_df.head()

In [ ]:
base_columns = set(['filename', 'Reports', 'attributes'])
MIRQI_LABELS = [c for c in mirqi_df.columns if c not in base_columns]

In [ ]:
len(chexpert_df), len(mirqi_df)

In [ ]:
df = chexpert_df.merge(mirqi_df, on='filename', suffixes=['_chx', '_mirqi'])
print(len(df))
df.head()

In [ ]:
reports_by_condition = defaultdict(set)

for index, row in chexpert_df.iterrows():
    filename = row['filename']
    report = row['Reports']
    labels = row[CHEXPERT_LABELS]

    tup = (index, filename, report)

    no_findings = labels['No Finding']
    
    if no_findings == 1:
        reports_by_condition['no-findings-1'].add(tup)
        if any(l != 0 for l in labels[1:-1]):
            # Exclude no-findings and support-devices
            reports_by_condition['inconsistent'].add(tup)
    else:
        if not any(l != 0 for l in labels[1:-1]):
            reports_by_condition['no-findings-absent'].add(tup)
    
    if all(l != 1 for l in labels):
        reports_by_condition['no-1s'].add(tup)
    
[(k, len(v)) for k, v in reports_by_condition.items()]

In [ ]:
l = list(reports_by_condition['no-findings-absent'])
l[:5]

In [ ]:
mirqi_df.loc[mirqi_df['filename'] == '256.xml'][MIRQI_LABELS]

In [ ]:
l = list(reports_by_condition['no-1s'])
l[:10]

In [ ]:
l = list(reports_by_condition['no-findings-1'])
l[:10]